# Turn to Horizontal

In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
%cd drive/MyDrive/DataHacks2021/

[Errno 2] No such file or directory: 'drive/MyDrive/DataHacks2021/'
/content/drive/MyDrive/DataHacks2021


In [56]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import DateOffset

In [57]:
df = pd.read_csv("combined.csv")
series_id = df["series_id"].unique()
date = df["date"].unique()
date_range = pd.date_range(date.min(), date.max(), freq='D')
date_range = [str(x)[:10] for x in date_range]

In [58]:
horizontal_df = pd.DataFrame(index=date_range, columns=series_id)

In [59]:
for row in range(len(df)):
    series_id = df.iloc[row].series_id
    date = df.iloc[row].date[:10]
    value = df.iloc[row].value
    horizontal_df.loc[date, series_id] = value
    if row % 10000 == 0:
        print(row)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000


In [ ]:
#horizontal_df.to_csv("1_raw_training_horizontal.csv")

# Missing value (not random)

In [60]:
series_info = pd.read_csv("series.csv")

In [61]:
series_info["frequency"].unique()

array(['Daily', 'Monthly', 'Weekly, Ending Saturday', 'Daily, 7-Day',
       'Weekly, Ending Thursday', 'Daily, Close',
       'Weekly, Ending Wednesday'], dtype=object)

In [62]:
df_copy = horizontal_df.copy(deep=True)

In [63]:
def monthly_fill(df):
    not_null_id = np.where(~df.isnull())
    for id in not_null_id[0]:
        value = df.iloc[id]
        start = pd.to_datetime(df.index[id]) + DateOffset(days=1)
        end = pd.to_datetime(df.index[id]) + DateOffset(months=1, days=-1)
        range = pd.date_range(start, end, freq='D')
        range = [str(x)[:10] for x in range]
        for d in range:
            if d in df.index:
                df.loc[d] = value
    return df

def weekly_fill(df):
    not_null_id = np.where(~df.isnull())
    for id in not_null_id[0]:
        value = df.iloc[id]
        start = pd.to_datetime(df.index[id]) + DateOffset(days=1)
        end = pd.to_datetime(df.index[id]) + DateOffset(weeks=1, days=-1)
        range = pd.date_range(start, end, freq='D')
        range = [str(x)[:10] for x in range]
        for d in range:
            if d in df.index:
                df.loc[d] = value
    return df

In [64]:
for col in df_copy.columns.values:
    freq = series_info.loc[series_info["series_id"]==col]["frequency"].values[0]
    if "Monthly" in freq:
        monthly_fill(df_copy[col])
    elif "Weekly" in freq:
        weekly_fill(df_copy[col])
    print(col)

AAA10Y
ASEANTOT
BAA10Y
BUSAPPWNSAUS
BUSAPPWNSAUSYY
CBUSAPPWNSAUS
CBUSAPPWNSAUSYY
CUUR0000SA0R
DEXCHUS
DEXUSEU
DEXUSUK
DFII10
DFII20
DFII30
DFII5
DFII7
DLTIIT
DPCREDIT
DPRIME
EECTOT
EFFR
EFFRVOL
FRGSHPUSM649NCIS
GVIPT50002S
GVIPT50030S
GVIPT51000S
GVIPT51100S
GVIPT51110S
GVIPT51120S
GVIPT51200S
GVIPT52000S
GVIPT52100S
GVIPT521A3S
GVIPT52300S
GVIPT54000S
GVIPT54100S
GVIPT54200S
GVIPT54220S
INFECTDISEMVTRACKD
IOER
IORR
MEDCPIM158SFRBCLE
MORTGAGE15US
MORTGAGE30US
OBFR
OBFRVOL
PCUADLVWRADLVWR
PCUAINFOAINFO
PCUAMUMAMUM
PCUARETTRARETTR
PCUASHCASHC
PCUASTDSVASTDSV
PCUATRADEATRADE
PCUATRANSATRANS
PCUATRNWRATRNWR
PCUATTDSVATTDSV
PCUAWHLTRAWHLTR
PCUOMFGOMFG
PCUOMINOMIN
SFTPAGRM158SFRBSF
SP500
T10YIE
TEDRATE
TLAACBW027NBOG
TLBACBW027NBOG
TSIFRGHT
UNRATE
WLEMUINDXD
SOFR
SOFRVOL


In [65]:
#df_copy.to_csv("2_raw_training_horizontal_without_weekend.csv")

# Drop weekends

In [ ]:
for id in df_copy.index:
    if pd.to_datetime(id).dayofweek in [5,6]:
        df_copy = df_copy.drop(id)

In [ ]:
df_copy.to_csv("3_combined.csv")